#학습 이전 스텝

## 데이터 처리

### 데이터, 라이브러리 불러오기

In [1]:
# 필요 라이브러리부터 설치할께요.
!pip install konlpy pandas seaborn gensim wordcloud python-mecab-ko wget

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 49.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 575.6/575.6 KB 29.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 KB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 15.9 MB/s eta 0:00:00
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9676 sha256=ab1839ac6035d2eb718b2a85cccbcaaee9c3654387eb6c282ae8ec230ef89699
  Stored in directory: /root/.cache/pip/wheels/04/5f/3e/46cc37c5d698415694d83f607f833f83f0149e49b3af9d0f38
Successfully built wget


In [2]:
from mecab import MeCab
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from IPython.display import display
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from wordcloud import WordCloud
from collections import Counter
import wget, os

In [3]:
!gdown https://drive.google.com/u/0/uc?id=1mK1GKocXxPzELN0jEUPLq6HOIoasC_Ve&export=download
data = pd.read_csv('/content/train.csv')

Downloading...
From: https://drive.google.com/u/0/uc?id=1mK1GKocXxPzELN0jEUPLq6HOIoasC_Ve
To: /content/train.csv
100% 1.31M/1.31M [00:00<00:00, 138MB/s]


In [4]:
label_dict = {
    '코드1': 0,
    '코드2': 0,
    '웹': 1,
    '이론': 2,
    '시스템 운영': 3,
    '원격': 4
}

data['label'] = data['label'].replace(label_dict)

### 데이터 전처리

In [5]:
#@title 전처리 통합 함수화
import re
import numpy as np
import mecab
import tensorflow as tf

def standardization_data(data, thres = 1.8, ngrams = 2, max_tokens = 35000, output_length = 50):

    global vectorized_layer
    global outlength
    outlength = output_length

    vectorized_layer = tf.keras.layers.TextVectorization(
        max_tokens=max_tokens,
        split="whitespace",
        ngrams=ngrams,  # Unigrams and bigrams
        output_mode='int',
        # pad_to_max_tokens=True,
        output_sequence_length=outlength,
        # standardize=standardization_data
    )

    data = np.array(data).reshape(-1)

    labels = ['plain', 'code', 'error', 'warning']
    n = len(labels)

    linecount = [[], [], [], [], []]
    lines = [[], [], [], []]

    mecab_instance = mecab.MeCab()
    
    for idx, rawtext in enumerate(data):
        texts = rawtext.split(sep='\n')
        texts = [text for text in texts if text != '']
        flags = [0] * n
        feature = [[], [], [], []]

        flag = 0

        for i, text in enumerate(texts):
            result = re.findall(r"[a-zA-Z]*.*[a-zA-Z]+\([^)]*\)|[a-zA-Z0-9_ \+-=!]+=[a-zA-Z0-9_ ]+|import[a-zA-Z0-9 ]+|#[\w ]+", text)
            res = ''.join(result)

            if not flag:
                if len(re.findall(r"원격", text)):
                    flag = 1
            
            if flags[0]:
                a = re.findall(r"[ㄱ-ㅎㅏ-ㅣ가-힣]+", text)
                if len(a):
                    b = re.findall(r"#+[ㄱ-ㅎㅏ-ㅣ가-힣 ]+", text)
                    if not len(b):
                        flags[0] = 0
            
            wa = re.findall(r"Warning[\w\(\) )*:]", text, re.I)
            if len(wa):
                flags[0] = 3
            
            er = re.findall(r"(Error|Exception|NotFound|Traceback)[\w\(\)) ]*(Traceback|:)", text, re.I)
            if len(er):
                flags[0] = 2

            if not flags[0]:
                if len(res) * thres > len(text):
                    flags[0] = 1

            feature[flags[0]].append(text)
            flags[flags[0]] += 1
        
        for _ in range(n):
            if len(feature[_]):
                lines[_].append(preprocess_text(" ".join(feature[_]), _, mecab_instance))
            else:
                lines[_].append("")
            
        for _ in range(n):
            if _:
                linecount[_].append(flags[_] / (i + 1))
            else:
                linecount[_].append((i + 1 - sum(flags[1:])) / (i + 1))
                
        linecount[4].append(flag)

    # labels = ['plain', 'code', 'error', 'warning']
    vectorized_layer.adapt(np.array(lines[0], dtype=object))

    global vocab_size
    vocab_size = vectorized_layer.vocabulary_size()

    return np.array(vectorized_layer(np.array(lines[0], dtype=object))), np.array(linecount).T

def preprocess_text(target = None, index = 0, method = mecab.MeCab()):
    anots = [
    'NNG', # 일반 명사
    'NNP', # 고유 명사
    # 'NNB', # 의존 명사
    # 'NNBC',# 단위를 나타내는 명사
    'NR',  # 수사
    'NP',  # 대명사
    'VV',  # 동사
    'VA',  # 형용사
    'VX',  # 보조용언
    'VCP', # 긍정 지정사
    'VCN', # 부정 지정사
    # 'MM',  # 관형사
    'MAG', # 일반 부사
    'MAJ', # 접속 부사
    # 'IC',  # 감탄사
    # 'JKS', # 주격 조사
    # 'JKC', # 보격 조사
    # 'JKG', # 관형격 조사
    # 'JKO', # 목적격 조사
    # 'JKB', # 부사격 조사
    # 'JKV', # 호격 조사
    # 'JKQ', # 인용격 조사
    # 'JC',  # 접속 조사
    # 'JX',  # 보조사
    # 'EP',  # 선어말어미
    # 'EF',  # 종결 어미
    # 'EC',  # 연결 어미
    # 'ETN', # 명사형 전성 어미
    # 'ETM', # 관형형 전성 어미
    # 'XPN', # 체언 접두사
    # 'XSN', # 명사 파생 접미사
    # 'XSV', # 동사 파행 접미사
    # 'XSA', # 형용사 파생 접미사
    'XR',  # 어근
    # 'SF',  # 마침표, 물음표, 느낌표
    # 'SE',  # 줄임표 ...
    # 'SSO', # 여는 괄호 (, [
    # 'SSC', # 닫는 괄호 ), ]
    # 'SC',  # 구분자 , · / :
    # 'SY',  # 기타 기호
    # 'SH',  # 한자
    # 'SL',  # 외국어
    # 'SN',  # 숫자
    ]

    if target == None:
        return ""
    
    try:
        # mecab_instance = mecab.MeCab()
        mepos = method.pos(target)
        word = [pos[0] for pos in mepos if pos[1] in anots]
        return " ".join(word)
    except Exception as e:
        print(f"Error processing text at index {index}: {e}")
        return ""

In [50]:
#@title Data Split 구현
import random as rd

def data_split(data, target = None, train_size = 0.8, thres = 1.8, ngrams = None, max_tokens = 30000, output_length = 50, val_size = 0, random_state = None):
    if target == None:
        target = list(data)[-1]

    rd.seed(random_state)

    nontest_idx = []
    
    for _ in data[target].unique():
        idx = set(data.loc[data[target] == _].index)
        nontest_idx.append( set(rd.sample(list(idx), int(len(idx) * train_size))) )

    train_idx = set()

    if val_size:
        for _ in nontest_idx:
            train_idx.update( set(rd.sample(list(_), int(len(_) * (1 - val_size)))) )
    
    nontest_idx = set([idx for sublist in nontest_idx for idx in sublist])

    data_size = len(data)

    test_idx = list(set(range(data_size)) - nontest_idx)
    val_idx = list(nontest_idx - train_idx)
    train_idx = list(train_idx)
    nontest_idx = list(nontest_idx)

    x = np.array(data.drop(target, axis=1))
    y = np.array(data[target])

    vector = standardization_data(data = x, thres = thres, ngrams = ngrams, max_tokens = max_tokens, output_length = output_length)

    data_size = vector[0].shape[0]

    x_train = []
    x_val = []
    x_test = []

    if val_size:
        for _ in vector:
            x_train.append(_[train_idx])
            x_val.append(_[val_idx])
            x_test.append(_[test_idx])

        y_train = y[train_idx]
        y_val = y[val_idx]
        y_test = y[test_idx]

        return x_train, x_val, x_test, y_train, y_val, y_test

    else:
        for _ in vector:
            x_train.append(_[nontest_idx])
            x_test.append(_[test_idx])

        y_train = y[nontest_idx]
        y_test = y[test_idx]

        return x_train, x_test, y_train, y_test

# split 테스트용 코드

# x_train, x_val, x_test, y_train, y_val, y_test = data_split(data, ngrams=1, val_size=0.2)
# np.unique(y_train, return_counts=True), np.unique(y_val, return_counts=True), np.unique(y_test, return_counts=True)

## 모델링

In [7]:
#@title Define Create Model

from tensorflow import keras

def create_model(vocab_size=None, output_length=None):
    reg = keras.regularizers.L1L2(l1 = 0.01, l2=0.02)
    if vocab_size == None:
        vocab_size=30000
    if output_length == None:
        output_length = 50
    keras.backend.clear_session()

    il1 = keras.layers.Input(shape=(output_length,), name='TextInput')
    il2 = keras.layers.Input(shape=(5,), name='InfoInput')

    embedding_layer = keras.layers.Embedding(vocab_size, 50)(il1)
    embedding_layer = keras.layers.Conv1D(128, 1)(embedding_layer)
    hl1 = keras.layers.Bidirectional(keras.layers.GRU(96, return_sequences=True))(embedding_layer)
    hl1 = keras.layers.Dropout(0.2)(hl1)
    hl1 = keras.layers.Conv1D(128, 2, 2)(hl1)
    hl1 = keras.layers.Bidirectional(keras.layers.GRU(128, return_sequences=True))(hl1)
    hl1 = keras.layers.BatchNormalization()(hl1)
    hl1 = keras.layers.Conv1D(256, 2, 2)(hl1)
    hl1 = keras.layers.Bidirectional(keras.layers.LSTM(196, return_sequences=True))(hl1)
    hl1 = keras.layers.Conv1D(512, 2, 2, kernel_regularizer=reg)(hl1)
    hl1 = keras.layers.GlobalAveragePooling1D()(hl1)

    hl2 = keras.layers.Dense(256, 'swish')(il2)

    hl = keras.layers.Concatenate()((hl1, hl2))
    hl = keras.layers.BatchNormalization()(hl)
    hl = keras.layers.Dense(512, 'relu')(hl)
    hl = keras.layers.Dropout(0.3)(hl)
    hl = keras.layers.Dense(128, 'relu')(hl)
    hl = keras.layers.BatchNormalization()(hl)
    ol = keras.layers.Dense(5, 'softmax')(hl)

    model = keras.models.Model((il1, il2), ol)
    model.compile('adam', keras.losses.sparse_categorical_crossentropy, metrics='accuracy')
    
    return model

# 모델 생성 및 컴파일
model = create_model()
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 TextInput (InputLayer)         [(None, 50)]         0           []                               
                                                                                                  
 embedding (Embedding)          (None, 50, 50)       1500000     ['TextInput[0][0]']              
                                                                                                  
 conv1d (Conv1D)                (None, 50, 128)      6528        ['embedding[0][0]']              
                                                                                                  
 bidirectional (Bidirectional)  (None, 50, 192)      130176      ['conv1d[0][0]']                 
                                                                                              

In [71]:
#@title model fitting 함수화
from sklearn.metrics import classification_report, confusion_matrix

def model_fitting(ngram=1, patience=8, stage=10, stage_patience=3, random_es_patience=True, val_size=0.2):
    print(f"ngrams = {ngram}")
    if val_size:
        x_train, x_val, x_test, y_train, y_val, y_test = data_split(data, ngrams=ngram, val_size=val_size)
    else: 
        x_train, x_test, y_train, y_test = data_split(data, ngrams=ngram)
    model = create_model(vocab_size, outlength)
    for _ in range(stage):
        if random_es_patience:
            es_patience = rd.randrange(patience, patience * 2)
        else:
            es_patience = patience
        print(f"Stage {_ + 1}: Early Stopping Patience is {es_patience}, fitting start...")
        es = keras.callbacks.EarlyStopping('val_loss', 0, es_patience, 0, restore_best_weights=True)
        lr = keras.callbacks.ReduceLROnPlateau('val_loss', 0.5, patience, 0)
        history = model.fit(x_train, y_train, epochs=10000, batch_size=256, callbacks=[es, lr], validation_data=(x_val, y_val), verbose=0)
        print(f"\t {history.epoch[-1] + 1} Epochs, Early Stopping at {history.epoch[-es_patience]}")
        print(f"\t Val Loss: {min(history.history['val_loss'])}, Val Accuracy: {min(history.history['val_accuracy']) * 100}%")
        if history.epoch[-1] ==  es_patience:
            stage_patience -= 1
        if stage_patience == 0:
            print(f"\n...Skipping Stage {_ + 2} ~ Stage {stage}\n")
            break
    result = model.evaluate(x_test, y_test, verbose = 0)
    print(f"Test Loss: {result[0]}, Test Accuracy: {result[1] * 100}%")
    print("="*100)
    y_pred = model.predict(x_test, verbose = 0)
    y_pred_max = y_pred.argmax(axis=1)
    print(classification_report(y_test, y_pred_max))
    print('='*100)
    print("코드, 웹, 이론, 운영, 원격")
    print(confusion_matrix(y_test, y_pred_max))

# 학습

In [72]:
#@title 1번 실험실
# ngrams = 1
model_fitting(1)

ngrams = 1
Stage 1: Early Stopping Patience is 9, fitting start...
	 74 Epochs, Early Stopping at 65
	 Val Loss: 1.0630805492401123, Val Accuracy: 29.29292917251587%
Stage 2: Early Stopping Patience is 11, fitting start...
	 13 Epochs, Early Stopping at 2
	 Val Loss: 1.1160401105880737, Val Accuracy: 69.5286214351654%
Stage 3: Early Stopping Patience is 9, fitting start...
	 10 Epochs, Early Stopping at 1
	 Val Loss: 1.1491897106170654, Val Accuracy: 69.02356743812561%
Stage 4: Early Stopping Patience is 11, fitting start...
	 12 Epochs, Early Stopping at 1
	 Val Loss: 1.0976808071136475, Val Accuracy: 69.69696879386902%
Stage 5: Early Stopping Patience is 8, fitting start...
	 9 Epochs, Early Stopping at 1
	 Val Loss: 1.137433648109436, Val Accuracy: 69.69696879386902%

...Skipping Stage 6 ~ Stage 10
Test Loss: 1.0558464527130127, Test Accuracy: 71.73620462417603%
              precision    recall  f1-score   support

           0       0.69      0.85      0.76       317
           1 

In [ ]:
#@title 2번 실험실
# ngrams = 2
model_fitting(2)

ngrams = 2
Stage 1: Early Stopping Patience is 11, fitting start...


In [ ]:
#@title 3번 실험실
# ngrams = 1 ~ 2
model_fitting((1, 2))

In [71]:
data.loc[data['label'] == 4]

,text,label
37,"말씀하신대로 하였으나, 이번에는 이런 오류가 발생합니다.\n\ncannot impo...",4
60,어떻게 삭제하는지 모르겠어요 ㅜ 원격지원 부탁드려도 될까요..,4
69,마지막으로 안내해주신 명령어대로 수행해봤는데 역시 에러메시지가 뜹니다 ㅠㅠ\n혹시 ...,4
101,https://113366.com?accessCode=107144링크로 접속 후 프...,4
271,"네트워크 오류 관련 알려주신 내용 확인해 봤는데 안됩니다,,,, 원격 부탁드려도 되...",4
...,...,...
3550,원격 요청 다시한번 드리니다!,4
3595,원격 지원이 실패한다고 뜹니다. 종료하고 다시 눌러도 안돼요\n다시 보내주시면 감...,4
3648,혹시 원격 지원 부탁드려도 될까요??\n,4
3651,key ppk 파일이 아무리 찾아도 보이지 않는데 혹시 원격지원 부탁드려도 될까요 ㅜ,4


In [85]:
data[data['text'].str.contains('원격')]['label'].value_counts()

4    99
1     3
0     2
3     1
Name: label, dtype: int64

In [88]:
data[data['text'].str.contains('지원')]['label'].value_counts()

4    50
1     6
2     4
3     3
0     2
Name: label, dtype: int64

In [89]:
data[data['text'].str.contains('제어')]['label'].value_counts()

4    5
0    3
3    1
Name: label, dtype: int64